# TME 7 : Mini-projet Detection de motifs


## Partie C : Recheche de pattern (motifs) en utilisant les algoritmes randomisés

Les algorithmes randomisés prendre des décisions aléatoire plutôt que déterministes.
l'algorithme s'exécute différemment à chaque fois. Ils sont couramment utilisés dans situations où aucun algorithme exact et rapide est connu. Nous allons implémenter deux algorithmes randomisés pour la recherche de motifs : Greedy Profile Motif Search et random Projections.


1\. Nous allons réutiliser les fonctions du TME6 pour générer t séquences artificielles de taille n, et implanter dans chaque séquence un motif de taille k à des positions aléatoires avec v substitutions choisies aléatoirement.

In [1]:
import random
import numpy as np
import copy

nuc = ('A', 'C', 'G', 'T')

k=7 #taille de motif
v=4 #nb de positions variable dans le motif
t=20 #nb de sequences
n=50 #longuer des sequence

def generateRandomSequence(n, I):
    seq_tmp = ""
    
    if I == True:
        for i in range(n):
            seq_tmp = seq_tmp + random.choice(nuc)
    else:
        for i in range(n):
            seq_tmp = seq_tmp + random.choice(nuc).lower()

    return seq_tmp
    

def generateRandomSequences(n, t):
    sequences = [generateRandomSequence(n, True) for i in range(t)]
   
    return sequences


def implantMotifVar(k, v, t, n):
    sequences = generateRandomSequences(n, t)
    motif = generateRandomSequence(k, True)
    print("Motif : ", motif)
    
    for i in range(len(sequences)):
        L_seq = list(sequences[i])
        L_motif = list(motif)
        for nb_mutation in range(v):
            index_mutation_motif = random.randint(0,len(motif) - 1)
            L_motif[index_mutation_motif] = random.choice(nuc)
        chn_motif_mute = "".join(L_motif)
        print(chn_motif_mute)
        L_seq.insert(random.randint(0, len(L_seq) - 1), chn_motif_mute)
        
        sequences[i] = "".join(L_seq)
    return sequences


In [2]:
#generate and implant variavel motifs


sequences = implantMotifVar(k, v, t, n)
print (sequences)


Motif :  GTCTTTC
GTACCTC
GTTTCTC
GGTTGTC
CGCCTTA
GACGTTC
GTCTTTT
ATATTTC
GTCTTTG
CTCTTCC
GTTTTCC
GGCCATC
TTCTCCC
ATATATC
CACTTAC
GTGTTCC
ATCGCTC
ATCTTAC
GGGTCTC
GTTTTTC
GTACTTC
['TGTACCTCCCACGTTACGCAGTAACCGTTTTGCTCGCAACAACTCTTTCAACTTATC', 'CCACTGTCGCGTACCATAGATACGGGATTCCTCCGAAGCGTGCAAGTTTCTCACGGG', 'TGAAGCAGCGACGGCGAACTATCATTGACTCGCTGTTGGGGTTGTCTCACGCAGTCA', 'GTGATCCCTACTCTCCCCGCCTTATGCAAGGACCCGAAAGGGAACCCATCCTTCGAC', 'TTGACTACTGCTATCAGTCAAAAAACGGTGACCTCCTAAGACGTTCCAAATCTCACA', 'TGCTATCCCCTACACGAGCCTGAAGGGATTGCCGGTCGTCTTTTTTGACACAACCAT', 'GGTCATCTTGAAAAACAAATATTTCGTAATTGCCTCGCCCGCTGGTGATGTCTTACA', 'ATTTCTCTCACCTGCATTCCGCGGCTGTGCGGGAGGGTCTTTGCAAACACCTAATCC', 'GATTTTGTTTATATGCCTTGTAACTCTCCGATACGTCGGAGCGAGGCTCTTCCCTGA', 'GCAAGTTTTCCAAAATCTTAGCCTAACACTGGCGCGCGACCCGATATAAGCTCCTTG', 'GGGCCATCTCTGTATCAACTGTTGGCGGGAGACTTGCGCTAACACCCGCGACCGCCC', 'CGACCTGGGCCTCTTTGAAGACCCGATTCACATATCCCGTTCTCCCGCCTCAGTCAA', 'TACAGGAGGGTTAACGTGAGTCAGTGCTCAGACGAAGGTCTAACGGGGATATATCGG', 'GGCCTACACTTACCTAGACCTGATGGAT

2\. Faire une fonction pour sélectionner des positions de départ aléatoirement s = (s1, …,st).


In [3]:
#creating vector s
#Get t random positions from 0 to n - k
def generateRandomS(sequences, k):
    s = []
    for i in range(t):
        s.append(random.randint(0, len(sequences[i]) - k))
    return s

s = generateRandomS(sequences, k)
print (s)
print(len(s))

[41, 9, 41, 32, 39, 18, 3, 4, 5, 9, 23, 46, 36, 23, 6, 15, 7, 40, 36, 33]
20


3\. Extraire les motifs en utilisant le vecteur s, et construire un profile. Attention, utiliser les pseudocount 1 pour éviter les probabilités zéro.


In [4]:
#extract sequences from seqs
def extractSeqs(s, seqs):
    motifs = []
    for i in range(len(s)):
        seq_tmp = ""
        for j in range(s[i], s[i] + k):
            seq_tmp = seq_tmp + seqs[i][j]
        motifs.append(seq_tmp)        
    
    return motifs
        
motifs = extractSeqs(s, sequences)
print (motifs)

#construct Profiles
def profile(motifs, k, t):
    q = len(nuc)
    PWM = np.ones((q, k))#pseudo count
    
    L = [x for x in nuc]
    
    dictionnaire = {}
    for i in range(len(L)):
        dictionnaire[L[i]] = PWM[i]
    
    for i in range(k):
        for j in range(len(motifs)):
            dictionnaire[motifs[j][i]][i] = dictionnaire[motifs[j][i]][i] + 1
            
    #print(dictionnaire)
    for i in range(q):
        PWM[i] = dictionnaire[L[i]]
    
    #print(PWM)
            
    return dictionnaire

def profile_from_dict(dic, k):#ici k est le v de la question 14
    q = len(nuc)
    PWM = np.ones((q, k))#pseudo count
    L = [x for x in nuc]
    
    dictionnaire = {}
    for i in range(len(L)):
        dictionnaire[L[i]] = PWM[i]
            
    for cle in dic:
        for i in range(k):
            dictionnaire[cle[i]][i] = dictionnaire[cle[i]][i] + dic[cle]
    
    for i in range(q):
        PWM[i] = dictionnaire[L[i]]
    
    return dictionnaire
    

pro = profile(motifs, k, t)
print(pro)

['ACTCTTT', 'CGTACCA', 'TTGTCTC', 'CCCGAAA', 'GACGTTC', 'CCTGAAG', 'CATCTTG', 'CTCTCAC', 'TGTTTAT', 'CCAAAAT', 'GGCGGGA', 'GCCTCAG', 'GGTCTAA', 'TGGATGG', 'GGGATCT', 'CATAGCG', 'TACGAAA', 'GCACCTG', 'ACGTAGG', 'GCTCTAT']
{'A': array([ 3.,  5.,  3.,  6.,  6., 10.,  6.]), 'C': array([8., 9., 7., 6., 6., 4., 4.]), 'G': array([8., 7., 5., 6., 3., 4., 8.]), 'T': array([5., 3., 9., 6., 9., 6., 6.])}


4\. Transformer le Profile en PWM. Il faut diviser chaque élément par la somme de ses colonnes.

In [5]:
#Somme des colonnes est toujours = t + len(nuc) = 24
def somme_colonnes(profile):
    colonne = []
    for i in range(k):
        s = 0
        for j in nuc:
            s = s + profile[j][i]
        colonne.append(s)
    return colonne

def PWM(profile):
    s_colonne = somme_colonnes(profile)
    profile_modifie = copy.deepcopy(profile)
    
    for i in range(k):
        for j in profile_modifie:
            profile_modifie[j][i] = profile_modifie[j][i] / s_colonne[i]
    
    return profile_modifie

pwm = PWM(pro)
print(pwm)
print(pro)

{'A': array([0.125     , 0.20833333, 0.125     , 0.25      , 0.25      ,
       0.41666667, 0.25      ]), 'C': array([0.33333333, 0.375     , 0.29166667, 0.25      , 0.25      ,
       0.16666667, 0.16666667]), 'G': array([0.33333333, 0.29166667, 0.20833333, 0.25      , 0.125     ,
       0.16666667, 0.33333333]), 'T': array([0.20833333, 0.125     , 0.375     , 0.25      , 0.375     ,
       0.25      , 0.25      ])}
{'A': array([ 3.,  5.,  3.,  6.,  6., 10.,  6.]), 'C': array([8., 9., 7., 6., 6., 4., 4.]), 'G': array([8., 7., 5., 6., 3., 4., 8.]), 'T': array([5., 3., 9., 6., 9., 6., 6.])}


5\. Faire une fonction pour calculer la probabilité d'un motif de taille k selon une PWM.


In [6]:
def probability(PWM, seq):
    """
    Hypothese : on suppose que la taille de seq est egale a la taille d'une valeur de PWM
    """
    prob = 1
    
    for i in range(len(seq)):
        prob = prob * PWM[seq[i]][i]
        
    return prob



6\. Faire une fonction pour calculer le pMostProbkmer d'une séquence, voir un exemple dans les slides de cours.

In [7]:
def tousLesKmers(ADN, k):
    """
    String -> List(String)
    Rend la liste de tous les k-mers present dans ADN
    """
    List_des_Kmer = []
    for i in range(len(ADN)-k+1):
        seq_i = ""
        for j in range(k):
            seq_i = seq_i + ADN[i+j]
        List_des_Kmer.append(seq_i)
    return List_des_Kmer

def pMostProbkmer(P, k, sequence):
    maxProb = 0
    s = 0 # position du k-mer le plus probable dans la sequence    
    L_kmer = tousLesKmers(sequence, k)
    
    for kmer in L_kmer:
        if probability(P, kmer) > maxProb:
            maxProb = probability(P, kmer)
            s = sequence.find(kmer)
    return s

s1 = pMostProbkmer(pwm, k, "GCCGGATACT")
print(s1)


2


7\. Faire une fonction pour obtenir les nouvelles positions de départ s = (s1, …,st), c’était à dire les positions qui contiens les k-mer le plus probables.

In [8]:
def getNewS(P, k, sequences):
    
    s = []
    for sequence in sequences:
        s.append(pMostProbkmer(P, k, sequence))
    
    return s

ns = getNewS(pwm, k, sequences)
print (ns)

[25, 38, 31, 18, 33, 18, 3, 10, 5, 40, 8, 46, 36, 41, 16, 19, 28, 46, 21, 33]


8\. La condition d’arrêt d’algorithme est le non changement du Profile d'une itération à l'autre, faire une fonction pour comparer deux Profils (matrice) et détecter le changement.

In [9]:
def changeProfile(P1, P2):
    
    changement = False
    
    L = [x for x in nuc]
    L2 = [["Pareil" for i in range(len(P1[L[0]]))] for j in range(len(P1))]
    dictionnaire = {}
    
    for i in range(len(L)):
        dictionnaire[L[i]] = L2[i]
        
    for nucleo in nuc:
        for i in range(len(P1[nucleo])):
            if P1[nucleo][i] != P2[nucleo][i]:
                dictionnaire[nucleo][i] = P2[nucleo][i]
                changement = True #Il y a un changement

    for definition in dictionnaire:
        print(definition, " ", dictionnaire[definition])
    
    
    return changement 
    

def getScore(P, k):
    sc = 0
    S = random.choice(nuc)
    
    for i in range(len(P[S])):
        colonne = []
        for cle in P:
            colonne.append(P[cle][i])
        sc = sc + max(colonne)
    
    return sc

def score(s, seqs):
    sc = 0
    profile1 = profile(extractSeqs(s, seqs), k, t)
    
    L_max_colonne = []
    
    for i in range(len(profile1["A"])):
        L_tmp = []
        for j in nuc:
            L_tmp.append(profile1[j][i])
        sc = sc + max(L_tmp)
    
    return (sc)

print(score(s, sequences))

ns = getNewS(pwm, k, sequences)
print (s, ns)

sc = getScore(pro, k)
print (sc)
#test = changeProfile(pwm, pwm)
#print(test)

59.0
[41, 9, 41, 32, 39, 18, 3, 4, 5, 9, 23, 46, 36, 23, 6, 15, 7, 40, 36, 33] [25, 38, 31, 18, 33, 18, 3, 10, 5, 40, 8, 46, 36, 41, 16, 19, 28, 46, 21, 33]
59.0


9\. Implementer l'algorithme ``GreedyProfileMotifSearch`` en utilisant les fonctions precedentes. 


In [10]:
def GreedyProfileMotifSearch(sequences, t, n, k):
    
    bestScore = 0
    s = generateRandomS(sequences, k)
    
    while score(s, sequences) > bestScore:
        pro = profile(extractSeqs(s, sequences), k, t)
        pwm = PWM(pro)
        bestScore = score(s, sequences)
        #print(bestScore)
            
        s = getNewS(pwm, k, sequences)
   
    return s, bestScore
    
        
s, bestScore = GreedyProfileMotifSearch(sequences, t, n, k)   
motifs = extractSeqs(s, sequences)
print (motifs)
        

['CCTCCCA', 'CCTCCGA', 'GCAGCGA', 'CCTACTC', 'CCTCCTA', 'TCCCCTA', 'CCCGCTG', 'GCTGTGC', 'TCTCCGA', 'TAAGCTC', 'CCCGCGA', 'CCTGGGC', 'TCAGTGC', 'GATGGTA', 'CCTGGTC', 'GCTCCTC', 'GCTCCTC', 'TCTGGTA', 'TATGTTC', 'TATACTA']


10\. Avez vous trouvez le motif implanté? Rexécuter l’algorithme plusieurs fois pour le trouver. 

Reponse: Trouvé en exécutant beaucoup de fois...

11\. Vous avez certainement observer que l’algorithme ne produire pas toujours la même sortie et que les résultats dépendent de la sélection aléatoire des positions de départ s. Pour augmenter nous chances de retrouvez les bons motifs, nous allons implémenter une version itérative ``GreedyProfileMotifSearchIte`` 
qui a chaque fois sauvegarde s et le score du profile associé à s, après I itération, l’algorithme renvoie le vecteur s ayant le plus grand score.

In [11]:
def maximumIndice(liste):
    maxi = liste[0]
    indice_max = 0
    longueur=len(liste)
    for i in range(longueur):
        if liste[i] >= maxi:
            maxi = liste[i]
            indice_max = i
    return indice_max

def GreedyProfileMotifSearchIte(sequences, t, n, k, It):
    Scores = []
    Positions = []
    
    for i in range(It):
        s, score = GreedyProfileMotifSearch(sequences, t, n, k)
        Scores.append(score)
        Positions.append(s)
    
    J = maximumIndice(Scores)
    
    return Positions[J], Scores[J]
    
s, bestScore = GreedyProfileMotifSearchIte(sequences, t, n, k, 100)
motifs = extractSeqs(s, sequences)
print (motifs)

['TCCCACG', 'TCTCACG', 'TCTCACG', 'TCTCCCC', 'TCTCACA', 'TATCCCC', 'TCGCCCG', 'TCTCACC', 'ACTCTCC', 'TCTTAGC', 'ACACCCG', 'TCTCCCG', 'TCTAACG', 'ACACCCG', 'ACTCAAG', 'TCGCTCC', 'TCTTACG', 'TTGCACC', 'TTTCGCG', 'TCGCTCT']


12\. Tester algorithme  ``GreedyProfileMotifSearchIte`` sur vos données de chipSeq.

In [12]:
k=7; v=1; t=20; n=80

def readFasta(genome, n):
    sequence = []
    file = open(genome, "r")
    sequence = []
    for s in file:
        if s[0] != ">":
            sequence.append(s.strip().upper())
    sequenceStr = "".join(sequence)
    
    sequence = [sequenceStr[i:i+n] for i in range(0, len(sequenceStr), n)]
    sequenceRet = [x for x in sequence if x]
    return sequenceRet

genome = "Sequence_by_Peaks_8.fasta" #path of your fasta seqs


sequencesChip   = readFasta(genome, n)
t = len(sequencesChip)
print (sequencesChip[8], t, n, k)
print(len(sequencesChip[len(sequencesChip) - 1]))

s, bestScore = GreedyProfileMotifSearchIte(sequencesChip, t, n, k, 100) #Patienter un peu lors de l execution...
print (bestScore, s)
motifs = extractSeqs(s, sequencesChip)
print (motifs)

ACCTTATGGTATTTATTTTCAAGATTCAGCTACACCTAATCAAGAAGGTATTTTAGAATTACATGATAATATTATGTTTT 912 80 7
12
4216.0 [50, 30, 33, 7, 11, 46, 17, 47, 10, 0, 62, 15, 18, 71, 72, 33, 68, 38, 53, 16, 67, 52, 70, 8, 30, 36, 49, 4, 3, 37, 30, 32, 18, 65, 4, 50, 17, 7, 60, 68, 18, 27, 63, 18, 13, 56, 58, 8, 65, 4, 3, 42, 56, 61, 70, 25, 67, 25, 12, 51, 26, 41, 47, 10, 73, 1, 21, 16, 53, 65, 12, 56, 56, 44, 56, 26, 65, 23, 25, 26, 71, 18, 71, 72, 21, 6, 18, 10, 53, 29, 51, 61, 1, 41, 51, 26, 42, 11, 72, 63, 1, 15, 34, 32, 24, 24, 33, 72, 0, 73, 35, 50, 66, 9, 32, 73, 34, 31, 56, 17, 12, 71, 66, 49, 9, 8, 3, 21, 70, 69, 1, 47, 34, 0, 12, 25, 14, 8, 67, 2, 50, 45, 73, 42, 1, 36, 8, 30, 27, 41, 55, 43, 56, 31, 32, 1, 60, 30, 57, 41, 13, 12, 68, 7, 12, 22, 58, 26, 69, 59, 70, 33, 63, 35, 28, 1, 66, 44, 5, 73, 51, 23, 64, 22, 47, 6, 17, 24, 44, 70, 69, 36, 63, 56, 61, 56, 70, 17, 0, 44, 14, 52, 43, 62, 55, 67, 23, 2, 33, 2, 49, 0, 45, 38, 32, 65, 53, 29, 26, 63, 4, 37, 1, 41, 41, 50, 38, 55, 41, 72, 38, 50, 61, 33,

13\. Nous allons implémenter l'algorithme ``randomProjection``. Faire une fonction pour générer une projection de l à k, voir un exemple dans les slides de cours

In [15]:
k=7 #taille de motif
v=4 #nb de positions variable dans le motif
t=20 #nb de sequences
n=50 #longuer des sequence

def getRandomFixePositions(k, n):
    listR = []
       
    for i in range(n):
        posHasard = random.randint(0, k-1)
        while posHasard in listR:
            posHasard = random.randint(0, k-1)
        listR.append(posHasard)
    return listR

lR = getRandomFixePositions(7, 4)
lR.sort()
print (lR)

def generateKey(positions, motif):
    m = ""
    for i in positions:
        m = m + motif[i]
    
    return m


def tousLesKmersList(seqs, k):
    """
    List(String) -> List(String)
    Rend la liste de tous les k-mers present dans les sequences de seqs
    """
    List_des_Kmer = []
    for sequence in seqs:
        for i in range(len(sequence) - k + 1):
            seq_i = ""
            for j in range(k):
                seq_i = seq_i + sequence[i+j]
            List_des_Kmer.append(seq_i)
            
    return List_des_Kmer

def removeLowComplexe(motifs, minrep):
    validMotif = []
    for mot in motifs:
        if mot.count('A') < minrep and mot.count('C') < minrep and mot.count('G') < minrep and mot.count('T') < minrep:
            validMotif.append(mot)
    return validMotif


def removeLowComplexePair(motifs):
    validMotif = []
    
    for cle in motifs:
        list_tmp = []
        list_tmp.append(cle[0])
        for i in range(2, len(cle), 2):
            if cle[i] == list_tmp[0]:
                list_tmp.append(cle[i])
        cpt_pos_pair = len(list_tmp)
        list_tmp = []
        list_tmp.append(cle[1])
        for i in range(3, len(cle), 2):
            if cle[i] == list_tmp[0]:
                list_tmp.append(cle[i])
        cpt_pos_impair = len(list_tmp)
        
        if not cpt_pos_pair == ((len(cle) // 2) + 1) and not cpt_pos_impair == (len(cle) // 2):
            validMotif.append(cle)
              
    return validMotif

def element_des_lists_valeur(dic):
    """
    Dict -> List[String]
    Rend la liste des motifs des valeur de chaque cle de dic
    """
    L = []
    for cle in dic:
        for mot in dic[cle]:
            L.append(mot)
    
    return L

minrep = 5

[0, 2, 4, 5]


14\. Implémenter l'algorithme ``randomProjection``.

In [16]:
def randomProjection(k, v, sequences):
    motifs  = {}; motifsSeq  = {}
    L_L_tuple = removeLowComplexe(tousLesKmersList(sequences, k), minrep)
    L_L_tuple = removeLowComplexePair(L_L_tuple) #On enleve les motifs peu complexes
    rfp = getRandomFixePositions(k, v)
    
    for mot in L_L_tuple:
        m = generateKey(rfp, mot)
        if m in motifs: 
            motifs[m] = motifs[m] + 1
        else:
            motifs[m] = 1
        if m in motifsSeq:
            motifsSeq[m].append(mot)
        else:
            motifsSeq[m] = [mot]
    
    return motifs, motifsSeq
#motifsSort = sorted(motifs, reverse=True, key=motifs.get)
motifs, motifsSeq = randomProjection(7, 4, sequences)
print (motifs)
#print(motifsSeq)
p_test = profile_from_dict(motifs, 4)
print(p_test)


{'TTCT': 3, 'ACCG': 3, 'CCTT': 4, 'CTCC': 3, 'ATGC': 4, 'CATC': 6, 'GCTA': 6, 'TGAC': 4, 'TCCG': 5, 'AAGT': 5, 'CGCT': 6, 'GTAA': 4, 'CAGC': 4, 'AATG': 2, 'GCAC': 3, 'TCAA': 4, 'AGCG': 6, 'ATCT': 5, 'CTGA': 6, 'CTTA': 2, 'GTTC': 6, 'TGTC': 3, 'TCTG': 9, 'TCCT': 5, 'GGTT': 4, 'CCCT': 4, 'CACC': 4, 'GCAT': 6, 'CAAC': 4, 'AACG': 7, 'ACAC': 5, 'CTAA': 4, 'ACCA': 4, 'ATTC': 9, 'CTCA': 9, 'TTTA': 3, 'TATT': 6, 'TACC': 9, 'TCAT': 5, 'CTAT': 6, 'AATC': 2, 'CCGC': 5, 'TGTA': 5, 'GCCC': 4, 'TGGT': 3, 'CTCG': 3, 'GAGT': 1, 'CCTC': 6, 'GCAG': 2, 'ATCG': 2, 'CAAT': 7, 'CGTA': 3, 'AAAC': 2, 'AGAA': 3, 'TGCG': 4, 'AGGA': 4, 'CAGT': 4, 'GTGA': 3, 'GTAC': 3, 'GCTG': 3, 'ACTG': 4, 'TTCG': 4, 'TCCA': 4, 'CGAT': 4, 'AGGC': 3, 'GGGA': 5, 'GAGG': 1, 'GGAG': 1, 'ATGG': 5, 'CGCC': 6, 'AGGT': 6, 'GCAA': 5, 'CGGA': 5, 'GCGC': 6, 'CGAA': 3, 'GGCG': 6, 'ACGC': 3, 'CGGG': 4, 'GACA': 3, 'GAGC': 2, 'CCAG': 4, 'GTAG': 3, 'AACC': 2, 'ATTG': 3, 'CCAA': 3, 'TATA': 4, 'ATCC': 4, 'TTAT': 6, 'AATT': 7, 'TCGC': 6, 'TTAA': 2

15\. Avez vous trouvez le motif implanté? Rexécuter l’algorithme plusieurs fois pour le trouver. 

reponse: Sans itérations, le resultat depend GRANDEMENT de generateRandomS(sequences, k)

16\. Implémenter la version itérative de l’algorithme ``randomProjection``. 

In [33]:
def randomProjIt(sequences, k, v, It):
    total_score=[]
    total_Motifs=[]
    total_score_dict = {} #dictionnaire des occurences
    tous_les_seqMotifs = []
    
    for i in range(It):
        motifss, seqsMotif = randomProjection(k, v, sequences)
        
        p_seqsMotif = profile(element_des_lists_valeur(seqsMotif), k, t)
        score_p_seqsMotif = getScore(p_seqsMotif, k)
        
        for cle in motifss:
            if cle in total_score_dict:
                total_score_dict[cle] = total_score_dict[cle] + motifss[cle]
            else:
                total_score_dict[cle] = motifss[cle]
        
        total_Motifs.append(motifss)
        total_score.append(score_p_seqsMotif)
        tous_les_seqMotifs.append(seqsMotif)
    
    J = maximumIndice(total_score)
        
    return total_score[J], total_Motifs[J], total_score_dict, tous_les_seqMotifs[J]

    #ou alors ...(ce que j'avais fait au debut...)

score, Motifss, total_score_dict, tous_les_seqMotifs = randomProjIt(sequences, 7, 4, 100)

print (score) #d'apres ce que j'avais compris depuis le forum...
print(total_score_dict) #d'apres ce que j'avais compris depuis les pdf de cours...
print(Motifss)

1662.0
{'TGCA': 395, 'CTCC': 398, 'CATC': 409, 'TCCA': 450, 'TCGC': 411, 'AATG': 276, 'CCTT': 550, 'GGAT': 292, 'CTCA': 449, 'ATGC': 338, 'GACG': 315, 'TCAC': 419, 'AGGA': 284, 'ACTG': 376, 'CAAT': 362, 'CGAA': 295, 'GTCA': 383, 'TACC': 428, 'TAGC': 363, 'TCTG': 475, 'TCTT': 375, 'GGTT': 321, 'CTTT': 378, 'CTCG': 424, 'GTTC': 436, 'CGCT': 388, 'ATCG': 345, 'CCAC': 337, 'AGAA': 221, 'ACCA': 386, 'CAAC': 351, 'TAAA': 216, 'CCCA': 299, 'TATC': 431, 'TACT': 418, 'TCTC': 472, 'ACTT': 452, 'ATCT': 422, 'CTAC': 458, 'TTAA': 299, 'AATC': 374, 'TATT': 288, 'CCAT': 445, 'CAGT': 383, 'GCTG': 343, 'CTCT': 525, 'GGGC': 268, 'TTCG': 456, 'ACGC': 326, 'CGTG': 336, 'AGCA': 320, 'TTCC': 497, 'AAAC': 290, 'GCTA': 325, 'ACAT': 327, 'GACA': 284, 'GTGC': 349, 'AAGG': 279, 'TCGG': 365, 'TGAG': 285, 'CGTA': 399, 'CGTT': 419, 'CTTC': 534, 'GCCT': 389, 'ACCC': 326, 'GCAG': 323, 'CCAA': 349, 'GGGA': 283, 'TACG': 351, 'GAGC': 316, 'ACGT': 358, 'AGCG': 359, 'GTAC': 364, 'TGAA': 247, 'TCGA': 376, 'TATG': 310, 'CAT

17\. Tester l'algorithme  ``randomProjection`` sur vos données de chipSeq. Puis générér le LOGO du motif trouvé

In [34]:
k=7; v=1; t=len(sequencesChip); n=80

score, Motifsss, total_score_dict, tous_les_seqMotifs = randomProjIt(sequencesChip, k, 4, 100)
print(score)
total_score_dict = sorted(total_score_dict.items(), reverse = True) #donne une liste de tuple ordonné
total_score_dict = sorted(total_score_dict,key = lambda x : x[1] , reverse = True) #liste triée selon les valeur de cles
print(total_score_dict)
#print(seqsMotif)


def concensus(P,k):
    concen = ""
    L_tmp = []
    
    for cle in P:
        for i in range(len(nuc)):
            L_tmp.append(P[cle])
    
    for i in range(k):
        maxi_tmp = []
        for j in range(len(L_tmp)):
            maxi_tmp.append(L_tmp[j][i])
        for cle in P:
            if P[cle][i] == max(maxi_tmp):
                concen = concen + cle
                break
    return concen
 
#test perso...  
m1, n1 = total_score_dict[0]

p_t = profile(tous_les_seqMotifs[m1], k, t)
conc = concensus(p_t, k)
print(conc)


123523.0
[('AATT', 51445), ('TATA', 50627), ('ATAT', 50525), ('TTAA', 50031), ('ATTA', 49078), ('TAAT', 48350), ('TATT', 45516), ('TTTA', 43659), ('TTAT', 43644), ('ATTT', 42738), ('ATAA', 41258), ('AAAT', 40198), ('AATA', 39511), ('TAAA', 38556), ('TGTA', 35399), ('AGTT', 35219), ('TTAG', 35100), ('TAGT', 34810), ('TATG', 34685), ('ATGT', 34441), ('TCTA', 34381), ('TACT', 34342), ('GATT', 33875), ('TTGA', 33816), ('GTTA', 33810), ('TTAC', 33800), ('AAGT', 33621), ('CATT', 33611), ('TATC', 33577), ('ATTG', 33535), ('AGTA', 33443), ('ATAG', 33420), ('GTAT', 33323), ('CTTA', 33188), ('ATGA', 33186), ('ACTT', 33159), ('AGAT', 33136), ('GTAA', 33129), ('TGAT', 33104), ('ATCT', 33008), ('TAAG', 32858), ('GAAT', 32844), ('TGAA', 32830), ('GATA', 32821), ('TCAT', 32741), ('TAGA', 32416), ('CTAA', 32411), ('ATAC', 32385), ('CTAT', 32359), ('ATTC', 32213), ('ACTA', 32178), ('ACAT', 32111), ('TTCA', 32075), ('AACT', 32043), ('AATG', 32029), ('TACA', 31906), ('CATA', 31795), ('TCAA', 31776), ('TG